In [1]:
%matplotlib inline
# imports required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynetlogo
import numpy as np
import sys
sys.path.append('../')

from LandLab_experiments.ecohydr_mod import EcoHyd

ModuleNotFoundError: No module named 'generate_uniform_precip'

In [ ]:
def setUpNetLogoModel(leadFarmers, desperation, jealousy, grace):
    # think this is for the GUI idk?
    sns.set_style("white")
    sns.set_context("talk")

    # starts a NetLogo link, point the netlogo home path to where you have it installed on your machine
    netlogo = pynetlogo.NetLogoLink(
        gui=False,
        netlogo_home="/Volumes/NetLogo 6.3.0/NetLogo 6.3.0"
    )

    # loads a .nlogo model from provided path
    netlogo.load_model("/Users/mabelwylie/Documents/NetLogo-MOSS-Python/CouplingRepo/TreesAndThat/Coupling/modelv2.nlogo")

     # sets globals
    globals = "update-globals " + str(leadFarmers) + " " + str(desperation) + " " + str(jealousy) + " " + str(grace)
    #netlogo.command(globals)
    
    # runs the model setup command
    netlogo.command("setup")

    return netlogo

In [ ]:
def reportsToDataFrame(netlogo):
    # gets field attributes and puts it in a data frame
    fieldAttributes = netlogo.report("get-info")
    sorted_list = sorted(fieldAttributes, key=lambda x: (-x[1],x[0]))
    fieldData = pd.DataFrame(columns=["who", "xcor","ycor","owner-id","implements-WSA", "yield"], data=sorted_list)
    return fieldData

In [ ]:
def convertWSAToNPArray(data):
    # sets bool into correct format to pass to hydrology model
    emptynp = np.empty((51,51))
    for i in range(0,51):
        for j in range(0,51):
            ycorFromIndex = i * -1 + 25
            xcorFromIndex = j - 25
            emptynp[i][j]= data.loc[(data['ycor'] == ycorFromIndex) & (data['xcor'] == xcorFromIndex)]["implements-WSA"].iloc[0]
    return emptynp

In [ ]:
def convertHydrologyToDF(hydrologyArray, data):
    # method converts the hydrology model output into a pandas dataframer
    flattenedArray = hydrologyArray.flatten()
    # 2 is the placeholder to multiply the biomass to get yield
    data["yield"] = flattenedArray * 2
    return data

In [ ]:
def fullModelRun(leadFarmers, desperation, jealousy, grace, fileSuffix):
    # sets up model
    netlogo = setUpNetLogoModel(leadFarmers, desperation, jealousy, grace)
    WSA_array = np.zeros((51,51))
    # Set 20 random values to 1
    #indices_to_set = np.random.choice(range(51 * 51), size=20, replace=False)
    #row_indices, col_indices = np.unravel_index(indices_to_set, (51, 51))
    #WSA_array[row_indices, col_indices] = int(1)
    Ecohyd_model = EcoHyd()

    # gets base field attributes and puts it in a data frame
    fieldData = reportsToDataFrame(netlogo)
    #fieldData['Year'] = -1
    
    for year in range(0,1):
        # run for 10 years

        # converts the usingWSA bool for each field into an NP array
        hydrologyInput = convertWSAToNPArray(fieldData)
        print("converted to NP array")

        # use dummy model
        # hydrologyOutput = hydrologyModelYear(hydrologyInput)

        biomass_harvest, SM_canic_end = Ecohyd_model.stepper(hydrologyInput)
        
        # trying to implement the hydrology model
        # input to this block will be the WSA np.array, with 0 or 1 corresponding to if the field is using WSA
        # the output of this block should be a np.array with biomass
        
        #Ecohyd_model = EcoHyd(WSA_array=hydrologyInput)
        #EcoHyd.stepper(Ecohyd_model)
        print(biomass_harvest)
        # converts the updated yields to the dataframe
        convertHydrologyToDF(biomass_harvest,fieldData)
        print("Converted output to df")

        # writes this new yield information to the netlogo implementation
        netlogo.write_NetLogo_attriblist(fieldData, "field")
        print("wrote to netlogo")

        # runs one step of social model
        netlogo.command("farming-year")
        print("farming command")

        # converts field data to a df
        fieldDataTemp = reportsToDataFrame(netlogo)
        #fieldDataTemp['Year'] = year
        fieldData = pd.concat([fieldData, fieldDataTemp], ignore_index=True)
        print("convertedtoDF")

        plt.imshow(np.reshape(hydrologyInput, (51,51)))
        plt.colorbar(extend = 'both')
        print(hydrologyInput)

        

## dummy hydrology model to test running
def hydrologyModelYear(input):
    print("One year yield")
    return np.zeros((51,51))

fullModelRun(20, 0.5, 0.5, 3, 1)

TypeError: EcoHyd.__init__() missing 1 required positional argument: 'WSA_array'

In [ ]:
netlogo = setUpNetLogoModel(20, 0.1, 0.2, 3)
df = reportsToDataFrame(netlogo)
df.loc[df["implements-WSA"] == 1]

,who,xcor,ycor,owner-id,implements-WSA,yield
1,896.0,25.0,22.0,487.0,1.0,100.0
16,1480.0,25.0,21.0,487.0,1.0,100.0
39,2427.0,25.0,14.0,799.0,1.0,100.0
62,1164.0,24.0,14.0,799.0,1.0,100.0
163,1742.0,22.0,-3.0,660.0,1.0,100.0
...,...,...,...,...,...,...
2419,1845.0,-22.0,-14.0,255.0,1.0,100.0
2446,2914.0,-22.0,-15.0,255.0,1.0,100.0
2470,2037.0,-23.0,-14.0,255.0,1.0,100.0
2486,2628.0,-23.0,-10.0,203.0,1.0,100.0
